### Environment Setup

In [2]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

### Interfacing with Model

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [8]:
# default model
llm.model_name

'gpt-3.5-turbo'

In [9]:
# custom model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#### Simplest Invoke

In [10]:
# simplest invoke
out = llm.invoke("Hello, who are you?")
print(out.content)

Hello! I’m an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and engage in conversation. How can I help you today?


In [11]:
print(f"{out.usage_metadata=}")
print(f"{out.response_metadata['model_name']=}")

out.usage_metadata={'input_tokens': 13, 'output_tokens': 37, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
out.response_metadata['model_name']='gpt-4o-mini-2024-07-18'


#### Streaming

We can stream via three ways:

1. stream
2. astream
3. astream_events

In [12]:
# stream
for t in llm.stream("Tell me a joke about Elon Musk"):
    print(t.content, end="")

Why did Elon Musk bring a ladder to SpaceX?

Because he wanted to reach new heights!

In [13]:
# an async call example, which is only possible in a jupyter notebook. otherwise asyncio.run() should be used in a script.
import asyncio

async def do_something():
    for i in range(1, 11):
       await asyncio.sleep(0.2)  # Simulate some asynchronous operation
       yield i
async for e in do_something():
    print(e,end="")

12345678910

In [14]:
# astream
async for t in llm.astream("Tell me a joke about Elon Musk"):
    print(t.content, end="")

Why did Elon Musk bring a ladder to SpaceX?

Because he wanted to reach new heights!

In [15]:
# astream_events
async for e in llm.astream_events("Tell me a joke about Elon Musk", version="v1"):
   if e["event"] == "on_chat_model_stream":
       print(e["data"]['chunk'].content,end="")

Why did Elon Musk bring a ladder to SpaceX?

Because he wanted to reach new heights!

#### Using Message Objects

Above we invoked LLM with raw string. There are other ways to invoke the model. One of them is to use message objects. This way, we can provide more information to the model. For example, we can provide a chat history to the model. We can also provide a system prompt to the model.

1. invoke (string): Takes in a text parameter as string, internally converts it to a HumanMessage and returns a AIMessage with content
2. invoke (messages as objects): Takes a list of specific message objects, returns a message. This way, system prompt or a chat history can be given.
3. invoke (messages as strings): Takes a list of tuples indicating role and its message ('system',"Your name is Ali")

Since the response is a AIMessage object, content is read to get the message

##### Single message

In [16]:
# single message object (should be a list in any case)
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage("Hello, who are you?")])

AIMessage(content="Hello! I’m an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and engage in conversation. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 13, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8884a8b-6ac4-40a7-8444-6267608ba698-0', usage_metadata={'input_tokens': 13, 'output_tokens': 37, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### Multiple messages

In [17]:
# multiple message objects including system prompt
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]

llm.invoke(messages)

AIMessage(content='My name is Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 27, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-81e93614-8640-43bd-93f2-5764848f622c-0', usage_metadata={'input_tokens': 27, 'output_tokens': 13, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
# easy way to create messages with multiple roles
llm.invoke([
    ('system',"Your name is Ali"),
    ('human',"What's your name?")
    ])

AIMessage(content='My name is Ali. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 19, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b45cc0c-740e-4465-ad01-8687e4a4357b-0', usage_metadata={'input_tokens': 19, 'output_tokens': 13, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates

Templates are used to generate prompts using input variables. This templates can be passed to the model together with their input variables. 

##### Template with single message

In [20]:
# a prompt template with single message type (becomes human message) and single input variable
from langchain_core.prompts import ChatPromptTemplate

template_str = """
Rate a product score of based on the following user comment from 0 to 5 where 5 is the maximum and 0 is the minimum.
comment: {comment}
"""
template = ChatPromptTemplate.from_template(template_str)
template

ChatPromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='\nRate a product score of based on the following user comment from 0 to 5 where 5 is the maximum and 0 is the minimum.\ncomment: {comment}\n'), additional_kwargs={})])

In [21]:
# filling the template and invoking llm
out = llm.invoke(template.invoke({"comment":"I hate this product. I'll throw it away. It's useless."}))
print(out.content)

Based on the user comment, I would rate the product a score of 0. The comment expresses strong dissatisfaction and indicates that the user finds the product completely useless.


In [22]:
# when there's one variable in the template, it can be passed directly.
out = llm.invoke(template.invoke("I hate this product. I'll throw it away. It's useless."))
print(out.content)

Based on the user comment, I would rate the product a score of 0.


##### Template with multiple variables

In [23]:
# Only possible through dictionary

template_str = """
Can {person1} have a conversation with {person2}?
Think step by step and answer.
"""
template = ChatPromptTemplate.from_template(template_str)
template

ChatPromptTemplate(input_variables=['person1', 'person2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['person1', 'person2'], input_types={}, partial_variables={}, template='\nCan {person1} have a conversation with {person2}?\nThink step by step and answer.\n'), additional_kwargs={})])

In [24]:
out = llm.invoke(template.invoke({'person1':"Jimi Hendrix","person2": "Barack Obama"}))
print(out.content)

To determine whether Jimi Hendrix could have a conversation with Barack Obama, we need to consider a few key points:

1. **Time Period**: Jimi Hendrix was active primarily in the 1960s and died in 1970. Barack Obama was born in 1961 and became a prominent political figure in the 2000s, eventually serving as President from 2009 to 2017. Since Hendrix passed away long before Obama became a public figure, they did not exist in the same time period.

2. **Physical Reality**: Given that Hendrix is deceased, a literal conversation between the two is impossible. Conversations require both parties to be alive at the same time.

3. **Hypothetical Scenarios**: If we consider a hypothetical scenario, such as a fictional or artistic representation (like a movie, book, or dream), one could imagine a conversation between the two. In such a scenario, the content of the conversation could explore topics like music, culture, politics, and social issues, reflecting their respective influences and experi

##### Template with multiple messages

Above we created a template from a string. This is treated as HumanMessagePromptTemplate by default. We can also create a SystemMessagePromptTemplate and combination of them.

In [25]:
# a prompt template with multiple role message types and multiple input variables
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# create the list of messages
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"),
    HumanMessagePromptTemplate.from_template("Users comment about the product: {comment}"),
])
template

ChatPromptTemplate(input_variables=['assistant_name', 'comment'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant_name'], input_types={}, partial_variables={}, template="You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='Users comment about the product: {comment}'), additional_kwargs={})])

In [26]:
# see the filled list of message objects
template.invoke({"assistant_name":"Bob","comment":"I hate this product. I'll throw it away. It's useless."}).to_messages()

[SystemMessage(content="You are a helpful assistant named Bob. You introduce yourself, including your name, and reply to users' product comments", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Users comment about the product: I hate this product. I'll throw it away. It's useless.", additional_kwargs={}, response_metadata={})]

In [27]:
out = llm.invoke(template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."}))
out.content

"Hi there! I'm Joe, and I'm here to help. I'm sorry to hear that you're not satisfied with the product. Could you share more about what specifically didn't meet your expectations? Your feedback is valuable, and I’d love to assist you in finding a solution!"

In [28]:
# easy way to create a prompt template with multiple role message types and multiple input variables
template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"),
    ("human","Users comment about the product: {comment}"),
])
template

ChatPromptTemplate(input_variables=['assistant_name', 'comment'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant_name'], input_types={}, partial_variables={}, template="You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='Users comment about the product: {comment}'), additional_kwargs={})])

In [29]:
out = llm.invoke(template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."}))
out.content

"Hi there! I'm Joe, and I'm here to help. I'm sorry to hear that you're not satisfied with the product. Could you share more about what specifically didn't meet your expectations? Your feedback is valuable, and I’d love to assist you in finding a solution!"

##### Template with placeholder

Templates can have placeholders like chat history. These placeholders can be filled with multiple messages.

In [30]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{chat_history}"),
    ]
)

template.invoke({"chat_history": [
                ("human","hi! my name is bob"),
                ("ai","Hello Bob! How can I assist you today?"),
            ]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi! my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})])

##### Template with Few-shot examples

FewShotPromptTemplate is used to build prompts with few-shot examples. This can be used to provide examples to the model.

In [32]:
from langchain_core.prompts import PromptTemplate,FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

template_str = """
User: {query}
AI: {answer}
"""
# create a prompt example from above template
example_template = PromptTemplate.from_template(template_str)


In [33]:
example_template.invoke(examples[0])

StringPromptValue(text="\nUser: How are you?\nAI: I can't complain but sometimes I still do.\n")

In [34]:
from langchain_core.prompts import PromptTemplate

prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

fs_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
)

In [35]:
print(fs_prompt_template.invoke({"query": "How is the wheather like?"}).to_string())

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: How is the wheather like?
AI: 


### Appendix

Above we filled the template with a dictionary and using invoke. Calling invoke creates a ChatPromptValue that can be passed to the model.
There are other ways to fill a template.

1. invoke: Parameters must be passed as dictionary. Result is same as format_prompt that can be passed directly to chat model.
2. format_messages: will create message list that can be passed directly to chat model
3. format_prompt: will create ChatPromptValue object that has the 'messages' list that can be also passed to chat model
4. format: will create a single string combining&stringifying messages that can be passed to llm model or chat model.
5. messages[i].prompt.format: will fill individual message in template

In [36]:
# invoke with dictionary resulting ChatPromptValue that can be passed to chat model.
template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})])

In [37]:
# format_messages with named arguments resulting message list that can be passed to chat model.
template.format_messages(assistant_name="Joe", comment="I'll throw it away. It's useless.")

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})]

In [38]:
# format_prompt with named arguments resulting ChatPromptValue that can be passed to chat model.
template.format_prompt(assistant_name="Joe", comment="I'll throw it away. It's useless.")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})])

In [39]:
# format with named arguments resulting raw string, but not usable.
template.format(assistant_name="Joe", comment="I'll throw it away. It's useless.")

'System: You are a helpful assistant.'

In [40]:
# we can format single message in the template
template.messages[0].prompt.format(assistant_name="Bob")

'You are a helpful assistant.'